In [1]:
import pandas as pd

In [2]:
def toGrid(lat, lon):
    row = 256
    column = 256
    seoul_start_lat = 37.698098
    seoul_start_lon = 126.799791
    seoul_end_lat = 37.472494
    seoul_end_lon = 127.142928
    lat_distance = seoul_end_lat - seoul_start_lat 
    lon_distance = seoul_end_lon - seoul_start_lon

    if seoul_end_lat > lat or lat > seoul_start_lat:
        return -1, -1
    if seoul_start_lon > lon or lon > seoul_end_lon:
        return -1, -1

    lat_distance2= lat - seoul_start_lat
    lon_distance2 = lon - seoul_start_lon
    row = (lat_distance2 * 255) / lat_distance
    col = (lon_distance2 * 255) / lon_distance

    return int(row), int(col)

In [3]:
# 환자 경로 파일, 환자 정보 병합 후 파일로 저장
patient_route = pd.read_csv('covid19/PatientRoute.csv')
patient_info = pd.read_csv('covid19/PatientInfo.csv')
patient_info = patient_info[["patient_id","global_num","sex","age","infection_case"]]

# sex, age 없는 경우 삭제
merged_route = pd.merge(patient_info, patient_route)
merged_route = merged_route[pd.notnull(merged_route['sex'])]
merged_route = merged_route[pd.notnull(merged_route['age'])]

# latitude, longitude → grid 정보
merged_route["row"] = ""
merged_route["col"] = ""

for i in range(len(merged_route)):
    latitude = merged_route['latitude'].values[i]
    longitude = merged_route['longitude'].values[i]
    row, col = toGrid(latitude, longitude)
    merged_route.iloc[i, merged_route.columns.get_loc('row')] = row
    merged_route.iloc[i, merged_route.columns.get_loc('col')] = col
    
# 서울 아닌 확진자 방문지 제거, 필요 없는 columns 제거
merged_route = merged_route.loc[merged_route['row'] != -1]
merged_route = merged_route[['patient_id', 'sex', 'age', 'infection_case', 'date', 'type', 'latitude', 'longitude', 'row', 'col']]

# 집단감염 value 수정
causes = merged_route['infection_case']
causes = causes.drop_duplicates(keep='last')
causes = causes.dropna()
causes = causes.tolist()
causes_stay = ["contact with patient", "overseas inflow", "etc"]

for i in range(len(merged_route)):
    cause = merged_route.iloc[i, merged_route.columns.get_loc('infection_case')]
    if cause in causes_stay: continue
    merged_route.iloc[i, merged_route.columns.get_loc('infection_case')] = "community infection"

# 파일로 저장
merged_route.to_csv("covid19/MergedRoute.csv", mode='w')